# Данные

In [1]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

X, y = make_regression(n_features=10, n_samples=10000)
model_sk =  LinearRegression()
model_sk.fit(X, y)
w = model_sk.coef_

X.shape, y.shape, w.shape

((10000, 10), (10000,), (10,))

# Собираем корпус под модель


In [2]:
from sklearn.model_selection import train_test_split, cross_val_score

In [59]:
from sklearn.base import RegressorMixin, BaseEstimator
import numpy as np

class AwesomeRegression(RegressorMixin, BaseEstimator):
    
    def __init__(self, tol=1e-6, lr=0.001, num_iter = 1000, penalty = None, alpha=1.0, l1_ratio=0.5, m = 2, metrics = 'MSE'):
        self.w = None
        self.tol = tol
        self.lr = lr     #шаг
        self.iter = 0
        self.num_iter = num_iter
        self.penalty = penalty
        self.alpha = alpha
        self.l1_ratio = l1_ratio
        self.m = m
        self.metrics = metrics
        
    def loss(self, X, y):
        if metrics == 'MSE':
            return ((y - X@self.w).T)@(y - X@self.w)/y.size
        if metrics == 'MAE':
            return sum(abs((y - X@self.w).T)/y.size)

    def grad_loss(self, X, y):
        if self.metrics == 'MSE':
            return -2*X.T@(y - X@self.w)
        if self.metrics == 'MAE':
            return (-X.T)@np.sign(y - X@w)
    
    def mse_loss_ridge(self, X, y):
        return ((y - X@self.w).T@(y - X@self.w)+self.alpha*self.w.T@self.w)/y.size
    
    
    def grad_loss_ridge(self, X, y):
        return -2*(((y-X@self.w).T)@X-self.alpha*(self.w).T)
    
    
    def mse_loss_lasso(self, X, y):
        return (((y - X@self.w).T)@(y - X@self.w)+self.alpha*np.linalg.norm(self.w,1))/y.size
    
    
    def grad_loss_lasso(self, X, y):
        return -2*((y-X@self.w).T)@X + self.alpha*np.sign(self.w.T)  #subgradient
    
    
    def mse_loss_elasticnet(self, X, y):
        return ((y - X@self.w).T@(y - X@self.w)/(2*y.size)+self.alpha*  \
                (self.l1_ratio*np.linalg.norm(self.w,1)+0.5*(1-self.l1_ratio)*self.w.T@self.w))/y.size\
    
    
    def grad_loss_elasticnet(self, X, y):
        return -2*X.T@(y - X@self.w)/(2*y.size)+self.alpha* \
                (self.l1_ratio*np.sign(self.w.T)+(1-self.l1_ratio)*self.alpha*w.T)
    
           
    def fit(self, X, y):
        self.w = np.random.normal(size=(X.shape[1]))
        
        if self.penalty == None:
            new_w = self.w - self.lr*self.grad_loss(X, y)

            while np.any(abs(new_w - self.w) > self.tol) and self.iter < self.num_iter:
                rand = np.random.randint(0,X.shape[0]-1,size=self.m)
                self.iter += 1
                self.w = new_w
                new_w = self.w - self.lr*self.grad_loss(X[rand], y[rand])
            
        
        if self.penalty == 'l1' or self.penalty == "l1":
            
            new_w = self.w - self.lr*self.grad_loss_lasso(X, y)

            while np.any(abs(new_w - self.w) > self.tol) and self.iter < self.num_iter:
                self.iter += 1
                self.w = new_w
                new_w = self.w - self.lr*self.grad_loss_lasso(X, y)
                
        if self.penalty == 'l2' or self.penalty == "l2":
            
            new_w = self.w - self.lr*self.grad_loss_ridge(X, y)

            while np.any(abs(new_w - self.w) > self.tol) and self.iter < self.num_iter:
                self.iter += 1
                self.w = new_w
                new_w = self.w - self.lr*self.grad_loss_ridge(X, y)
        
        if self.penalty == 'elasticnet' or self.penalty == "elasticnet":
            
            new_w = self.w - self.lr*self.grad_loss_elasticnet(X, y)

            while np.any(abs(new_w - self.w) > self.tol) and self.iter < self.num_iter:
                self.iter += 1
                self.w = new_w
                new_w = self.w - self.lr*self.grad_loss_elasticnet(X, y)
            

    def fit_SGD(self, X, y):  
        self.w = np.random.normal(size=(X.shape[1]))
        
        new_w = self.w - self.lr*self.grad_loss(X, y)

        while np.any(abs(new_w - self.w) > self.tol) and self.iter < self.num_iter:
            
            rand = np.random.randint(0,X.shape[0]-1)
            self.iter += 1
            self.w = new_w
            new_w = self.w - self.lr*self.grad_loss(X[rand], y[rand])
    
    def fit_formula(X, y):
        self.w = np.linalg.inv(X.T@X)@(X.T@y)
     
    def predict(self, X):
        return X@self.w

In [34]:
(-2*X.T@(y - X@w))

array([ 5.66143177e-10,  3.23145317e-11,  6.51840259e-10, -1.39460415e-09,
        8.96463400e-10,  4.68664955e-10, -3.08390951e-11, -1.72530483e-09,
        1.56690477e-09,  1.01260835e-09])

In [58]:
(-X.T)@np.sign(y - X@w)

array([ 1404.85983692,    92.93882489,  1693.29671773, -3506.15657903,
        2238.92562467,  1146.29125193,  -162.35304991, -4265.01570879,
        3960.65689975,  2531.00219585])

In [33]:
np.sign(y - X@w)

array([-1., -1., -1., ..., -1., -1.,  1.])

# fit with SGD

In [ ]:
def fit(self, X, y):  
        self.w = np.random.normal(size=(X.shape[1]))
        
        new_w = self.w - self.lr*self.grad_loss(X, y)

        while np.any(abs(new_w - self.w) > self.tol) and self.iter < self.num_iter:
            
            rand = np.random.randint(0,X.shape[0]-1)
            self.iter += 1
            self.w = new_w
            new_w = self.w - self.lr*self.grad_loss(X[rand], y[rand])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [60]:
model = AwesomeRegression(num_iter=100000,m=2,metrics='MAE')
model.fit(X_train,y_train)

model.iter

54

In [61]:
model.w

array([-2.4920827 ,  0.89421528, -0.59415466,  3.28268326, -0.9823396 ,
       -1.77006323, -0.55548609,  2.64698244, -3.38108839, -3.36257387])

In [62]:
model_sk.coef_

array([18.28467543, 45.92406008, 39.72233445, 88.51367472, 51.29294693,
       29.60089231, 24.97097067, 67.17237199,  1.66021256, 25.7986071 ])

In [12]:
model_sk.score(X_test,y_test)

1.0

## Задание 1 (добить линейную регрессию): 

__а)__ Добавить в обучение параметр `num_iter`, чтобы не возникало такого, что из-за неадекватно подобранного параметра `tol` цикл работает вечно. Установить параметр по дефолту в `1000` шагов.

__б)__ Добавить в модель возможность вводить l1 и l2 регуляризаторы (elstic_net), для этого придется выписать на бумажке фукнцию потерь и найти её дифференциал. Для l2 проблем быть не должно, для l1 придется быть поосторожнее с нулём. 

__в)__ Реализовать вместо текущего обычного градиентного спуска стохастический градиентный спуск по мини-батчам. Надо на каждой итерации отбирать $m$ случайных наблюдений и делать шаг градиентного спуска по ним. Параметр $m$ задаётся при объявлении модели.

__г)__ Сейчас обучение идёт с помощью функции потерь MSE. Хочется, чтобы была возможность выбора. 

- Добавьте такую возможность в виде функций потерь MAE
- Добавьте MAPE

__HINT:__ подсказка как добавить MAPE прямо через MAE: https://yadi.sk/i/WpIWG_PYeQkLVQ

- Добавьте функцию потерь Хубера

__HINT:__ про неё можно почитать либо в [главе про функции потерь Дьяконова](https://alexanderdyakonov.files.wordpress.com/2018/10/book_08_metrics_12_blog1.pdf) либо в [первой главе Магнуса Катышева Персецкого.](https://yadi.sk/i/K8P9rL1eDK0Dqg)

Склёрновская реализация такой модели [тут.](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.HuberRegressor.html)

- Придумайте свою функцию потерь с логарифмами такую, чтобы она была робастной к выбросам. 


__д)__ Протестировать модель на сгенерированном датасете. Сравнить результаты с пакетной реализацией. Подумать каких ещё приблуд можно засунуть внутрь модели. 

## Задание 2 (логистическая регрессия): 

Нужно реализовать аналогичный класс для обучения логистической регрессии. Должен быть точно такой же интерфейс, но другая модель. 

- В качестве функции потерь возьмите `logloss`. Добавьте опциональные ругуляризаторы, модель учите стохастическим градиентным спуском. Все производные найдите руками с помощью матричного диффериенцирования. Сравните работу модели с пакетной реализацией и возрадуйтесь.